In [95]:
# https://stackoverflow.com/questions/33289820/noun-phrases-with-spacy

In [1]:
from coref import *

In [2]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 17.4MB/s eta 0:00:011
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.0-cp37-none-any.whl size=98072933 sha256=8eef9f4f668dae59a89ef821d02ae505e2654d1d1bb3ea6a749475c696b050bf
  Stored in directory: /private/var/folders/qj/vhjml8kj1tl0shjyxg0rfq5c0000gn/T/pip-ephem-wheel-cache-_13z7upu/wheels/5f/3e/c9/36dd6e13b449fd84cd1f94b72dfbc559daf09f53dbf4e697a3
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [51]:
import spacy
nlp = spacy.load("en_core_web_md")

In [52]:
input_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/a9.input'
key_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/a9.key'

In [53]:
reader = ReadInput(input_path)
list_of_sentences = reader.getListOfSentences()
# print(list_of_sentences)
full_text = reader.getText()
# print(full_text)

In [54]:
sentence_dict = getSentenceDict(list_of_sentences)
cluster_head_dict = getClusterHeads(sentence_dict)

In [15]:
doc = nlp('This is a test sentence. Exam is what is written.')

In [29]:
doc[3], doc[6], doc[10]

(test, Exam, written)

In [34]:
doc[6].similarity(doc[3])

0.59551954

In [71]:
type(doc[6])

spacy.tokens.token.Token

In [72]:
type(doc[3])

spacy.tokens.token.Token

In [148]:
nlp('CRASHES').similarity(nlp('ACCIDENTS'))

0.6178732115598177

In [153]:
list(nlp('F14-CRASHES').noun_chunks)

[]

In [55]:
def getRemText(sent_id, sentence_dict):
    rem_text = {}
    for sid in sentence_dict.keys():
        if(sid >= sent_id): 
            rem_text[sid] = sentence_dict[sid]
    sorted_rem_text = dict(sorted(rem_text.items(), key = lambda x:x[0]))
    return sorted_rem_text

In [56]:
# def getSimilarityScore(cluster_head_name,noun_phrase):
#     ## When NP will be used, then partial_ratio will be used
# #     return fuzz.partial_ratio(cluster_head_name.lower(),noun_phrase.lower_)

In [156]:
def getCorefDict(sentence_dict, cluster_head_dict, threshold):
    
    ## Initialise a coref_dict 
    coref_dict = {}
    
    for cluster_id, cluster_value in cluster_head_dict.items():
#         print(cluster_id)
        current_cluster_sent_id = cluster_value[0]        
        current_cluster_head = cluster_value[1]
        
        remaining_text = getRemText(current_cluster_sent_id, sentence_dict)

        for sid, sentence in remaining_text.items():
#             print(sid, sentence)
#             print('*'*8)
            clean = re.compile('<COREF .*?>.*?</COREF>') ## removing all coref tags from the current sentence
            clean_sentence = re.sub(clean, '', sentence)
#             print(clean_sentence)
#             print('*'*16)
#             clean_sentence = clean_sentence.replace('-',' ')
#             print(clean_sentence)

            doc = nlp(clean_sentence)
#             print(list(doc.noun_chunks))
            word_embedding_score_list = []
            
            for np in doc.noun_chunks:
                
                ## If the NP is a pronoun, ignore it. 
                ## We are handling pronouns by Hobb's algorithm
                if(len(np) == 1 and np[0].pos_ == 'PRON'):
                    continue
                
                similarity_score = nlp(current_cluster_head)[0].similarity(np)
#                 print(similarity_score)
                
                               
                word_embedding_score_list.append(list([current_cluster_head, sid, np, similarity_score]))
            
#             print(word_embedding_score_list)
            if(len(word_embedding_score_list) == 0):
                continue
            scores_list = list(map(lambda x: x[3], word_embedding_score_list))
            max_score = max(scores_list)
            max_index = scores_list.index(max_score)

            max_word_embedding_score_list = word_embedding_score_list[max_index]


            if(max_score > threshold):
                if(cluster_id not in coref_dict.keys()):
                    coref_dict[cluster_id] = max_word_embedding_score_list
                else:
                    coref_dict[cluster_id].append(max_word_embedding_score_list)
                    
                    
#                     print(cluster_id, current_cluster_head, word, sid, similarity_score)
#                     if(cluster_id not in coref_dict.keys()):
#                         coref_dict[cluster_id] = [list([current_cluster_head, sid, np, similarity_score])]
#                     else:
#                         coref_dict[cluster_id].append(list([current_cluster_head, sid, np, similarity_score]))
    return coref_dict

In [127]:
list_of_sentences

['<S ID="0">nyt960222.0269</S>\n',
 '<S ID="1">A2555</S>\n',
 '<S ID="2">BC-<COREF ID="X0">F14-CRASHES</COREF>-BLOOM</S>\n',
 '<S ID="3">@amp;LR;</S>\n',
 '<S ID="4"><COREF ID="X1">02-22</COREF></S>\n',
 '<S ID="5">BC-F14-CRASHES-BLOOM</S>\n',
 '<S ID="6"><COREF ID="X2">NORTHROP GRUMMAN F-14S</COREF> TO STAND DOWN FOR REVIEW OF ACCIDENTS</S>\n',
 '<S ID="7">(For use by New York Times News Service clients)</S>\n',
 '<S ID="8">By Bill Arthur</S>\n',
 '<S ID="9">c.1996 <COREF ID="X3">Bloomberg Business News</COREF></S>\n',
 '<S ID="10">Washington, Feb. 22 (Bloomberg) -- The <COREF ID="X4">Navy</COREF> ordered its Northrop Grumman Corp. F-14s out of the skies for <COREF ID="X5">three days</COREF> while it investigates three recent F-14 crashes.</S>\n',
 '<S ID="11">The action came after an F-14A fighter crashed today in the <COREF ID="X6">Persian Gulf</COREF>.</S>\n',
 '<S ID="12">The two crew members ejected and were rescued and returned to the aircraft carrier USS Nimitz with minor injur

In [155]:
coref_dict = getCorefDict(sentence_dict, cluster_head_dict,0.5)

X0


/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/py

/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/py

/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/py

X1


...

/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)


X13
X14


/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)


X15


/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)


In [157]:
coref_dict

{'X2': ['NORTHROP GRUMMAN F-14S',
  10,
  its Northrop Grumman Corp. F-14s,
  0.6381004,
  ['NORTHROP GRUMMAN F-14S', 29, now Northrop Grumman, 0.74871886],
  ['NORTHROP GRUMMAN F-14S', 33, Northrop Grumman shares, 0.6855497]],
 'X4': ['Navy',
  12,
  the Navy,
  0.8559929,
  ['Navy', 16, the Navy, 0.8559929],
  ['Navy', 17, the Navy, 0.8559929],
  ['Navy', 19, Oceana Naval Air Station, 0.5122978],
  ['Navy', 20, the Navy's first-line fighter aircraft, 0.55538195],
  ['Navy', 22, The Navy, 0.8559929],
  ['Navy', 26, The Navy, 0.8559929],
  ['Navy', 27, Navy jet fighter aircraft, 0.64572954],
  ['Navy', 30, the Navy, 0.8559929],
  ['Navy', 31, the Navy, 0.8559929],
  ['Navy', 32, Naval Operations, 0.5237841]],
 'X5': ['three days',
  10,
  three recent F-14 crashes,
  0.62302005,
  ['three days', 12, The two crew members, 0.7305959],
  ['three days', 13, its two crew members, 0.7194855],
  ['three days', 14, three people, 0.8329172],
  ['three days', 16, the recent mishaps, 0.5386264],


In [100]:
coref_dict

{'X2': [['NORTHROP GRUMMAN F-14S',
   10,
   its Northrop Grumman Corp. F-14s,
   0.6381004],
  ['NORTHROP GRUMMAN F-14S', 29, now Northrop Grumman, 0.74871886],
  ['NORTHROP GRUMMAN F-14S', 33, Northrop Grumman shares, 0.6855497]],
 'X4': [['Navy', 12, the Navy, 0.8559929],
  ['Navy', 16, the Navy, 0.8559929],
  ['Navy', 16, all F-14 aircraft, 0.50042427],
  ['Navy', 16, a Navy spokesman, 0.7121709],
  ['Navy', 17, the Navy, 0.8559929],
  ['Navy', 19, Oceana Naval Air Station, 0.5122978],
  ['Navy', 20, the Navy's first-line fighter aircraft, 0.55538195],
  ['Navy', 22, The Navy, 0.8559929],
  ['Navy', 26, The Navy, 0.8559929],
  ['Navy', 27, Navy jet fighter aircraft, 0.64572954],
  ['Navy', 30, the Navy, 0.8559929],
  ['Navy', 31, the Navy, 0.8559929],
  ['Navy', 32, Naval Operations, 0.5237841]],
 'X5': [['three days', 10, three recent F-14 crashes, 0.62302005],
  ['three days', 12, The two crew members, 0.7305959],
  ['three days', 13, its two crew members, 0.7194855],
  ['three d

In [91]:
coref_dict

{'X0': [[5, BC-F14-CRASHES-BLOOM], [10, three recent F-14 crashes]],
 'X1': [[34, NYT-02-22-96 1221EST]],
 'X2': [[13, an F-14D],
  [14, an F-14A],
  [17, all F-14s],
  [29, Northrop Grumman],
  [33, Northrop Grumman shares]],
 'X4': [[12, the Navy],
  [16, the Navy],
  [17, the Navy],
  [20, the Navy's first-line fighter aircraft],
  [22, The Navy],
  [26, The Navy],
  [27, Navy jet fighter aircraft],
  [30, the Navy],
  [31, the Navy]],
 'X5': [[32, three days]],
 'X6': [[18, the Persian Gulf]],
 'X7': [[18, the standdown]],
 'X8': [[18, Geisen], [27, Geisen], [28, Geisen], [32, Geisen]],
 'X9': [[20, The F14]],
 'X15': [[33, Northrop Grumman shares]]}

In [52]:
reader_o = ReadInput(key_path)
ans = reader_o.getListOfSentences()

In [53]:
ans

['<COREF ID="X0">F14-CRASHES</COREF>\n',
 '{5} {F14-CRASHES} {CRASHES}\n',
 '{6} {ACCIDENTS} {ACCIDENTS}\n',
 '{10} {three recent F-14 crashes} {crashes}\n',
 '{16} {the recent mishaps involving F-14As and Ds} {mishaps}\n',
 '{17} {these mishaps} {mishaps}\n',
 '{32} {the three recent crashes} {crashes}\n',
 '\n',
 '<COREF ID="X1">02-22</COREF>\n',
 '{10} {Feb. 22} {Feb. 22}\n',
 '{11} {today} {today}\n',
 '{34} {02-22-96} {02-22-96}\n',
 '\n',
 '<COREF ID="X2">NORTHROP GRUMMAN F-14S</COREF>\n',
 '{10} {its Northrop Grumman Corp. F-14s} {F-14s}\n',
 '{16} {all F-14 aircraft} {F-14 aircraft}\n',
 '{17} {all F-14s} {F-14s}\n',
 '{22} {about 330 F-14s} {F-14s}\n',
 '{29} {The planes} {planes}\n',
 '\n',
 '<COREF ID="X3">Bloomberg Business News</COREF>\n',
 '{10} {Bloomberg} {Bloomberg}\n',
 '\n',
 '<COREF ID="X4">Navy</COREF>\n',
 '{10} {it} {it}\n',
 '{12} {Navy} {Navy}\n',
 '{16} {Navy} {Navy}\n',
 '{17} {Navy} {Navy}\n',
 '{22} {Navy} {Navy}\n',
 '{26} {Navy} {Navy}\n',
 '{27} {Navy} {